# SQL

SQL is a relational database management system that provides access to different tables. In data science, it is used to send, retrieve, and organize data with queries.

SQL is most often used to extract subsets of interest from large datasets or to join/merge datasets in order to gather relevant information stored in different tables.   We can also perform basic operations in SQL like filtering rows and generating descriptive statistics.

## Learning Outcomes

At the end of this section you should be comfortable with the SQL syntax used to: 
- Select columns
- Filter rows
- Aggregate data

## A Conceptual Introduction

### User Interface

The user interface of SQL consists of three main areas:
- The left panel shows the available tables in a tree-like structure.
- The main panel is where queries are written.
- The bottom panel displays the data once the query is executed.

However, we won't be using the actual SQL interface. Instead, we can write SQL queries using Python (i.e. within Google Collab) with packages like `sqlite3` or `sqlalchemy`.

![SQL UI](https://img.informer.com/screenshots/134/134493_1.jpg)

### Relational Database Structure

As mentioned earlier, tables stored in SQL are usually **related** by a particular column. Oftentimes the linking column will be some sort of unique customer or user ID. This is helpful when you want to **join** or **merge** tables together. You will learn this later in the course.

![relations](https://i.stack.imgur.com/h4XRK.png)

## Connecting to a Database

In [1]:
import sqlite3
import pandas as pd

In [2]:
# Establish a connection to a database
conn = sqlite3.connect('employee.db')
# Create a cursor
c = conn.cursor()

### Creating a Data Table (not required to know)

In [3]:
# WARNING: ONLY RUN THIS ONCE PER SESSION! OTHERWISE, YOU WILL RECEIVE AN ERROR

# Initialize the table
c.execute("""CREATE TABLE employees (
  first text,
  last text, 
  age integer, 
  pay integer)""")

# Populate the table
c.execute("""INSERT INTO employees (
  first, last, age, pay)
  VALUES 
  ('Anthony', 'Apple', 30, 50000),
  ('Bella', 'Banana', 22, 60000),
  ('Chase', 'Cranberry', 16, 10000),
  ('David', 'Dragonfruit', 50, 80000)""")

# Commit the changes 
conn.commit()

In [4]:
# Write our first query to check if the data exists
c.execute("SELECT * FROM employees")
# Print the output of the query
print(c.fetchall())

[('Anthony', 'Apple', 30, 50000), ('Bella', 'Banana', 22, 60000), ('Chase', 'Cranberry', 16, 10000), ('David', 'Dragonfruit', 50, 80000)]


We can easily reformat this output into a dataframe using the `pandas.read_sql_query` function:

In [5]:
pd.read_sql_query("SELECT * FROM employees", conn)

,first,last,age,pay
0,Anthony,Apple,30,50000
1,Bella,Banana,22,60000
2,Chase,Cranberry,16,10000
3,David,Dragonfruit,50,80000


We have successfully created a sample dataset using SQL's `INSERT` statement. Now we will learn how to select, filter, and aggregate data with SQL. You have learned similar operations in Python, so we will compare the code that produces identical outputs side-by-side.

## Selecting Columns

Columns can be selected using SQL's `SELECT` statement. In the query above, we ran `SELECT * FROM employees`. The `*` selects all columns by default. `FROM` specifies which dataset we want to use. In this case it is named `employees`. If we wanted to only select the first three columns in `employees`, we can choose which columns by specifying their names separated by commas:

In [6]:
pd.read_sql_query("SELECT first, last, age FROM employees", conn)

,first,last,age
0,Anthony,Apple,30
1,Bella,Banana,22
2,Chase,Cranberry,16
3,David,Dragonfruit,50


Now let us compare the SQL `SELECT` statement with how we would select columns in Python.

In [7]:
# Create an identical "employees" dataset in pandas
employees_df = pd.DataFrame(data={'first': ['Anthony','Bella','Chase','David'],
                                  'last': ['Apple','Banana','Cranberry','Dragonfruit'],
                                  'age': [30, 22, 16, 50],
                                  'pay': [50000, 60000, 10000, 80000]})

In [8]:
employees_df[['first','last','age']]

,first,last,age
0,Anthony,Apple,30
1,Bella,Banana,22
2,Chase,Cranberry,16
3,David,Dragonfruit,50


Write a SQL query to select the `age` and `pay` columns from `employees`.

In [9]:
# Your code here
pd.read_sql_query("SELECT age, pay FROM employees", conn)

,age,pay
0,30,50000
1,22,60000
2,16,10000
3,50,80000


## Filtering Rows

Filtering can be achieved in SQL with a `WHERE` statement. It is positioned **after** the `FROM` statement and can include all of the classic comparison operators we know and love such as `=`, `<`, `>`, `<=`, and `>=`.

In [10]:
pd.read_sql_query("SELECT first, last FROM employees WHERE age > 20", conn)

,first,last
0,Anthony,Apple
1,Bella,Banana
2,David,Dragonfruit


Here we've selected the first and last names of all employees older than 20. If we wanted to filter on a variable that contains strings rather than numeric, we would use `LIKE` as the comparison operator. More information can be found [here](https://www.w3schools.com/SQL/sql_like.asp).

In Python, we can filter and select with square brackets:

In [11]:
employees_df[employees_df['age'] > 20][['first','last']]

,first,last
0,Anthony,Apple
1,Bella,Banana
3,David,Dragonfruit


Write a SQL query to filter for observations with `pay` greater than or equal to $\$50,000$.

In [12]:
# Your code here 
pd.read_sql_query("SELECT * FROM employees WHERE pay >= 50000", conn)

,first,last,age,pay
0,Anthony,Apple,30,50000
1,Bella,Banana,22,60000
2,David,Dragonfruit,50,80000


## Aggregate Functions

Just like in Python, we can get descriptive statistics for variables of our choosing using SQL functions like `AVG`, `MIN`, `MAX`, and more.

In [13]:
pd.read_sql_query("SELECT MAX(age), AVG(pay) FROM employees", conn)

,MAX(age),AVG(pay)
0,50,50000.0


In Python, we simply call the `.max()` or `.mean()` function on the pandas dataframe column of interest:

In [14]:
print(employees_df[['age']].max())
print(employees_df[['pay']].mean())

age    50
dtype: int64
pay    50000.0
dtype: float64


Write a SQL query to find the minimum `age` of employees and the `SUM` of all their salaries.

In [15]:
# Your code here
pd.read_sql_query("SELECT MIN(age), SUM(pay) FROM employees", conn)

,MIN(age),SUM(pay)
0,16,200000


In [16]:
# IMPORTANT: Close the connection to the database when you are done querying
conn.close()